In [78]:
import argparse

import torch
import uuid

import numpy as np

from torch_geometric.nn import Node2Vec
from torch_geometric.utils import erdos_renyi_graph

import os
import csv

import gph
import gtda
from gtda import diagrams
from gtda import homology

import matplotlib.pyplot as plt

import sys
# appending a path
sys.path.append('../rawos')
import metrics


In [79]:
def total_persistence(diagram, p=2):
    """Calculate total persistence of a persistence diagram."""
    return np.sum(np.power(np.abs(np.diff(diagram[:, 0:2])), 2))

def get_dimension(diagrams, dim=0):
    """Get specific dimension of persistence diagram."""
    # This reads a little bit weird because of the idiosyncratic way of
    # handling masks. We first extract all triples in all time steps
    # that match the specified dimension. *Then* we remove the dimension
    # information (since it is spurious) and reshape the array to
    # account for the original number of time steps again. Whew!
    T = len(diagrams)
    mask = diagrams[..., 2] == dim
    diagrams = diagrams[mask][:, :2].reshape(T, -1, 2)

    return diagrams, T


In [89]:
## import files
c = 5
d = 16
l = 5
n = 10

max_dim = 1

"""c = 5
d= 5
l=10
n=10"""



filename = 'er'
filename += '-c'+str(c)
filename += '-d'+ str(d)
filename += '-l'+str(l)
filename += '-n'+ str(n)

## get filenames according to filter
path = '../results'
#path = '../rawos'
files = []
for i in os.listdir(path):
    if os.path.isfile(os.path.join(path,i)) and filename in i:
        files.append(i)

## load embeddings
Emb_list= []
dim = 0
for i,file in zip(range(len(files)),files): 
    emb = np.loadtxt(path+'/'+file, delimiter="\t")
    diam = metrics.diameter(emb)
    Emb_list.append(emb/diam)
    if emb.shape[0] > dim:
        dim = emb.shape[0]
        

Embs = np.zeros((len(Emb_list),dim, d)) 
for i in range(len(Emb_list)):
    Embs[i,:,:] = Emb_list[i] 

ph = gtda.homology.VietorisRipsPersistence(metric='euclidean', max_edge_length=np.inf, homology_dimensions=tuple(range(max_dim+1)), coeff=2, infinity_values=None, n_jobs=None).fit_transform(Embs)


In [93]:
Total_pers = []
for i in range(ph.shape[0]):
    Total_pers.append(total_persistence(ph[i,:,:],2))
print(Total_pers)

[9.371290405921307, 10.520334260273302, 13.100107757770068, 13.702381461918115, 10.836404870206657, 8.648231650589691, 12.827913595219947, 11.22237666626538, 10.15325338617463, 8.122722488466898]


In [94]:
## write files
